# Dealing with nested data in pandas DataFrame
Sometimes the data inside a cell in a dataframe is nested (lists of lists, lists of dict, dict of lists, etc). If it is a complex datastructure, we will not be able to analyze this using the usual pandas techniques. In this post, I will demostrate a way to explode them into rows or columns, so that it can be analyzed.

First some imports.

In [1]:
import pandas as pd
import numpy as np
import ast

C:\Users\Sandhya\Anaconda3\envs\PythonData\lib\site-packages\numpy\core\__init__.py:29: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Sandhya\Anaconda3\envs\PythonData\lib\site-packages\numpy\.libs\libopenblas.BNVRK7633HSX7YVO2TADGR4A5KEKXJAW.gfortran-win_amd64.dll
C:\Users\Sandhya\Anaconda3\envs\PythonData\lib\site-packages\numpy\.libs\libopenblas.CSRRD7HKRKC3T3YXA7VY7TAZGLSWDKW6.gfortran-win_amd64.dll
  stacklevel=1)


For this demo, I am going to use one file from the [Kaggle movies dataset](https://www.kaggle.com/rounakbanik/the-movies-dataset).

In [2]:
credits_df = pd.read_csv("data/credits.csv")

In [3]:
credits_df.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


Notice that each `cast` value is a list of dictionaries. Printing out just one to clearly look at it. After some thought, I decide to split the `list` into rows, so that each movie has multiple rows with cast details in a `dict`. After that, I'll split the `dict` to create multiple columns into each row.

In [4]:
credits_df["cast"][0]

"[{'cast_id': 14, 'character': 'Woody (voice)', 'credit_id': '52fe4284c3a36847f8024f95', 'gender': 2, 'id': 31, 'name': 'Tom Hanks', 'order': 0, 'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'}, {'cast_id': 15, 'character': 'Buzz Lightyear (voice)', 'credit_id': '52fe4284c3a36847f8024f99', 'gender': 2, 'id': 12898, 'name': 'Tim Allen', 'order': 1, 'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'}, {'cast_id': 16, 'character': 'Mr. Potato Head (voice)', 'credit_id': '52fe4284c3a36847f8024f9d', 'gender': 2, 'id': 7167, 'name': 'Don Rickles', 'order': 2, 'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg'}, {'cast_id': 17, 'character': 'Slinky Dog (voice)', 'credit_id': '52fe4284c3a36847f8024fa1', 'gender': 2, 'id': 12899, 'name': 'Jim Varney', 'order': 3, 'profile_path': '/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg'}, {'cast_id': 18, 'character': 'Rex (voice)', 'credit_id': '52fe4284c3a36847f8024fa5', 'gender': 2, 'id': 12900, 'name': 'Wallace Shawn', 'order': 4, 'profile_path': '/oGE6JqPP2xH4t

In [5]:
type(credits_df["cast"][0])

str

The value of the cell is not really a list of dictionaries! It just looked that way, it is really a `str`, so it has to be really converted to a list of dictionaries first. Fortunately, python has `ast` (Abstract Syntax Trees) module. The `literal_eval` function safely evaluates the string to create the python list of dicts. This is saved back to the column. Note the dataframe still "looks" the same, but the data type is different

In [6]:
credits_df['cast'] = credits_df['cast'].apply(ast.literal_eval)


In [7]:
type(credits_df["cast"][0])

list

In [8]:
type(credits_df["cast"][0][0])

dict

Time to create new rows for each of the list element. From [this](https://stackoverflow.com/a/42012264) answer, a new dataframe is created. `cast` is split into rows, and `id` is repeated to fill the new rows.

In [9]:
%%time
credits_df2 = pd.DataFrame({'id':np.repeat(credits_df['id'].values, credits_df['cast'].str.len()),
                        'cast':np.concatenate(credits_df['cast'].values)})

Wall time: 176 ms


In [10]:
credits_df2.head()

,id,cast
0,862,"{'cast_id': 14, 'character': 'Woody (voice)', ..."
1,862,"{'cast_id': 15, 'character': 'Buzz Lightyear (..."
2,862,"{'cast_id': 16, 'character': 'Mr. Potato Head ..."
3,862,"{'cast_id': 17, 'character': 'Slinky Dog (voic..."
4,862,"{'cast_id': 18, 'character': 'Rex (voice)', 'c..."


In [11]:
credits_df2.shape

(562474, 2)

Nice! Now its time to split the cast details to create new columns. Here [this answer](https://stackoverflow.com/a/38231651) was of help. However, this takes a long time to run on the 560k rows

In [12]:
%%time
credits_df3 = credits_df2['cast'].apply(pd.Series)

Wall time: 4min 24s


In [13]:
credits_df3['id'] = credits_df2['id']

In [14]:
credits_df3.head()

,cast_id,character,credit_id,gender,id,name,order,profile_path
0,14,Woody (voice),52fe4284c3a36847f8024f95,2,862,Tom Hanks,0,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg
1,15,Buzz Lightyear (voice),52fe4284c3a36847f8024f99,2,862,Tim Allen,1,/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg
2,16,Mr. Potato Head (voice),52fe4284c3a36847f8024f9d,2,862,Don Rickles,2,/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg
3,17,Slinky Dog (voice),52fe4284c3a36847f8024fa1,2,862,Jim Varney,3,/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg
4,18,Rex (voice),52fe4284c3a36847f8024fa5,2,862,Wallace Shawn,4,/oGE6JqPP2xH4tNORKNqxbNPYi7u.jpg


In [15]:
credits_df3.shape

(562474, 8)

Note that the operations were only done on the `cast` column. The `crew` column is yet to be tackled. Assuming the credits are given to 10 cast and 10 crew members for each movie in the original dataframe, if we explode `crew` column too, it will result in 10x10 = 100 rows for each movie. So the initial  45k rows will turn into 4.5 million rows - which may be too big. So the data has to be reduced - it will depend on the application. 

If we are building a recommender system, one possibility is to limit the `cast` data to `order` 0 to 3. Also, the `crew` data may be limited to director, writer and producer. 

If we are trying to figure out top 3 actors with the most number of films, we can drop `crew` column altogether.

If we are trying to figure out top 3 actor - director combinations, then the rest of the information can be dropped.

There are a lot of possibilities, and its easy to get into trouble with large amounts of data.